<a href="https://colab.research.google.com/github/siddadel/kalidas/blob/main/Dialogue_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!pip install transformers --quiet
!git clone https://github.com/allenai/comet-atomic-2020
!pip install -r ./comet-atomic-2020/requirements.txt --quiet
!wget https://storage.googleapis.com/ai2-mosaic-public/projects/mosaic-kgs/comet-atomic_2020_BART.zip
!unzip comet-atomic_2020_BART.zip

     |████████████████████████████████| 3.1 MB 7.0 MB/s 
     |████████████████████████████████| 3.3 MB 32.9 MB/s 
     |████████████████████████████████| 895 kB 66.4 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
Cloning into 'comet-atomic-2020'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 166 (delta 42), reused 84 (delta 27), pack-reused 56
Receiving objects: 100% (166/166), 7.15 MiB | 13.82 MiB/s, done.
Resolving deltas: 100% (48/48), done.
     |████████████████████████████████| 43 kB 323 kB/s 
     |████████████████████████████████| 90 kB 6.2 MB/s 
     |████████████████████████████████| 313 kB 51.7 MB/s 
     |████████████████████████████████| 9.1 MB 17.0 MB/s 
     |████████████████████████████████| 379 kB 44.1 MB/s 
     |████████████████████████████████| 829 kB 51.8 MB/s 
     |████████████████████

In [3]:
from google.colab import drive
drive.mount('/content/drive')
data_root = "/content/drive/MyDrive/ANLP21/scripts_txt"
#data_root = "/content/drive/MyDrive/ANLP21/scripts_sample"
output_dir= "/content/drive/MyDrive/ANLP21/exp"

Mounted at /content/drive


In [17]:
import pandas as pd
import random
import os
import re
import spacy
from collections import Counter
from joblib import Parallel, delayed
import pandas as pd
nlp = spacy.load("en_core_web_sm")

In [13]:
import sys
sys.path.insert(1,r'./comet-atomic-2020/models/comet_atomic2020_bart')
from generation_example import Comet
print("model loading ...")
comet = Comet("comet-atomic_2020_BART")
comet.model.zero_grad()
print("model loaded")

model loading ...
model loaded


In [14]:
relations = ["CapableOf",  "Desires",  "MotivatedByGoal",  "xAttr", "xNeed", "xReact", "xReason", "xWant"]

vocabulary = set()

film_bags = {}

def register_in_bag(film, token, relation):
    if film not in film_bags:
      film_bags[film] = {}
    if(relation not in film_bags[film]):
      film_bags[film][relation] = set()
    film_bags[film][relation].add(token)
    vocabulary.add(token)
    
def process_results(film, results, relations):
    for i in range(len(relations)):
      relation = relations[i]
      for result in results[i]:
        tokens = result.split(' ')
        for token in tokens:
          if token not in nlp.Defaults.stop_words:
            register_in_bag(film, token, relation)

log = []
def infer(film, item, relations):
    queries= ["{} {} [GEN]".format(item, relation) for relation in relations]
    results = comet.generate(queries, decode_method="beam", num_generate=5)
    if film not in log:
      print(film, end=",")
      log.append(film)
    process_results(film, results, relations)

In [15]:
df1 = pd.read_csv("/content/drive/MyDrive/ANLP21/data.csv")

In [18]:
df1.apply(lambda row: infer(row['film'], row['item'], relations), axis=1)

KeyboardInterrupt: ignored